In [187]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso


data = pd.read_csv('C:/Users/JP/Documents/School/Practicum/Github/Practicum/data/dataset2.csv')

df = pd.DataFrame(data)
df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')
df = df.set_index('Date')
df=df[df.index>'2017-05-30']

train=df[df.index<'2017-10-05']

features = ['TBondsOpenValue', 'UnemploymentValue', 'BTCOpenTM1', 'BTCOpenTM2',\
        'BTCOpenTM3', 'BTCOpenTM4', 'BTCOpenTM5', 'BTCOpenTM6', 'BTCOpenTM7','GDP','SandPValue',\
            'ETHOpenTM1', 'ETHOpenTM2','PrevHigh','PrevHigh2','PrevHigh3','PrevHigh4','PrevLow','PrevLow2','PrevLow3',\
            'PrevLow4','PrevVolTo','PrevVolTo2','PrevVolTo3','PrevVolTo4','PrevVolFrom','PrevVolFrom2','PrevVolFrom3',\
            'PrevVolFrom4','PrevSP','PrevSP2','PrevSP3','PrevSP4',\
        'ETHOpenTM3', 'ETHOpenTM4', 'ETHOpenTM5', 'ETHOpenTM6', 'ETHOpenTM7']


train=train.dropna()


dtrain = xgb.DMatrix(train.loc[:, features].values, \
                     label = train.loc[:, 'ETHOpen'].values)

params = {}
params['booster']  = 'gbtree'
params['objective'] = 'reg:linear'
params['max_depth'] = 6
params['subsample'] = 0.8
params['colsample_bytree'] = 0.8
params['silent'] = 1
params['eval_metric'] = 'rmse'
num_round = 50
eval_list  = [(dtrain,'train')]

train['Date'] = train.index.values

print('Training xgb model:')
bst = xgb.train(params, dtrain, num_round, eval_list)

print('Train Ridge Regression:')
lr = Ridge()
lr.fit(train.loc[:, features].values, \
       train.loc[:, 'ETHOpen'].values)

print('Training Lasso Regression:')
lassoreg = Lasso(alpha=.001,normalize=True, max_iter=1e7)
lassoreg.fit(train.loc[:, features].values,train.loc[:, 'ETHOpen'].values)
 
test1 = df[df.index>='2017-10-05']
test1=test1.dropna()

while True:
    dtest1 = xgb.DMatrix(test1[features].values)
    xgb_pred = bst.predict(dtest1)
    lr_pred = lr.predict(test1[features].values)
    lasso_pred = lassoreg.predict(test1.loc[:, features].values)
    test1['ETHOpenRidgexgb1'] = 0.2*xgb_pred+0.8*lr_pred
    test1['ETHOpenRidge1'] = lr_pred
    test1['ETHOpenxgb1'] = xgb_pred
    test1['ETHOpenLasso1'] = lasso_pred


    target = train['ETHOpen']
    
    done = 1
    
    if done:
        print("Prediction: {}".format(test1[['ETHOpen','ETHOpenRidgexgb1','ETHOpenRidge1','ETHOpenxgb1','ETHOpenLasso1']]))
        break
        
        
   
test1 = pd.DataFrame(test1[['ETHOpen','ETHOpenRidgexgb1','ETHOpenRidge1','ETHOpenxgb1','ETHOpenLasso1']])

Training xgb model:
[0]	train-rmse:201.232
[1]	train-rmse:143.012
[2]	train-rmse:102.264
[3]	train-rmse:74.8139
[4]	train-rmse:53.7264
[5]	train-rmse:39.2052
[6]	train-rmse:29.1174
[7]	train-rmse:22.1461
[8]	train-rmse:16.7908
[9]	train-rmse:13.4215
[10]	train-rmse:10.3724
[11]	train-rmse:8.36058
[12]	train-rmse:7.17471
[13]	train-rmse:5.87937
[14]	train-rmse:4.84781
[15]	train-rmse:4.02787
[16]	train-rmse:3.39152
[17]	train-rmse:2.95932
[18]	train-rmse:2.57113
[19]	train-rmse:2.35651
[20]	train-rmse:2.0432
[21]	train-rmse:1.71312
[22]	train-rmse:1.53414
[23]	train-rmse:1.36991
[24]	train-rmse:1.15787
[25]	train-rmse:1.04841
[26]	train-rmse:0.916114
[27]	train-rmse:0.816167
[28]	train-rmse:0.721434
[29]	train-rmse:0.624753
[30]	train-rmse:0.541245
[31]	train-rmse:0.494745
[32]	train-rmse:0.458097
[33]	train-rmse:0.408072
[34]	train-rmse:0.374002
[35]	train-rmse:0.333974
[36]	train-rmse:0.30529
[37]	train-rmse:0.275639
[38]	train-rmse:0.250836
[39]	train-rmse:0.226013
[40]	train-rmse:0.

In [71]:
test.tail()

,ETHOpen,ETHHigh,ETHLow,volumeto,volumefrom,TBondsOpenValue,IndexFutures,InflationRateValue,UnemploymentValue,FedFundRateValue,...,PrevVolFrom3,PrevVolFrom4,PrevSP,PrevSP2,PrevSP3,PrevSP4,ETHOpenRidgexgb,ETHOpenRidge,ETHOpenxgb,ETHOpenLasso
Date,,,,,,,,,,,,,,,,,,,,,
2017-10-08,311.26,313.52,308.13,6.780159e+07,217688.05,152.28125,93.570,2.233,4.2,1.16,...,202892.42,231043.01,2548.6,2535.7,2531.5,2526.5,305.591141,307.145887,299.372162,308.117369
2017-10-09,309.49,310.14,286.52,1.407532e+08,471243.74,152.28125,93.570,2.233,4.2,1.16,...,265830.27,202892.42,2548.6,2548.6,2535.7,2531.5,305.574517,305.701118,305.068115,306.876066
2017-10-10,296.95,305.95,292.28,8.047586e+07,268423.20,152.06250,93.315,2.233,4.2,1.16,...,269276.51,265830.27,2548.6,2548.6,2548.6,2535.7,292.145294,292.208023,291.894379,293.340465
2017-10-11,298.46,305.12,297.18,6.724668e+07,223439.16,152.21875,93.240,2.233,4.2,1.16,...,217688.05,269276.51,2543.8,2548.6,2548.6,2548.6,301.105066,301.657646,298.894745,302.910014
2017-10-12,302.86,308.77,301.65,1.303575e+08,426811.81,152.18750,92.705,2.233,4.2,1.16,...,471243.74,217688.05,2543.3,2543.8,2548.6,2548.6,301.433652,300.949565,303.369995,302.369040


In [33]:


from sklearn.linear_model import RandomizedLasso

print('Training Randomized Lasso')
rlasso = RandomizedLasso(alpha=0.0001,normalize=True,max_iter=1e7)
rlasso.fit(train.loc[:, features].values,train.loc[:, 'ETHOpen'].values)
 
print("Features sorted by their score:")
print(sorted(zip(map(lambda x: round(x, 4), rlasso.scores_), \
                 features), reverse=True))

  

Train Randomized Lasso Regression:


D:\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:377: RuntimeWarning: overflow encountered in true_divide
  g1 = arrayfuncs.min_pos((C - Cov) / (AA - corr_eq_dir + tiny))
D:\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:377: RuntimeWarning: overflow encountered in true_divide
  g1 = arrayfuncs.min_pos((C - Cov) / (AA - corr_eq_dir + tiny))
D:\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:377: RuntimeWarning: overflow encountered in true_divide
  g1 = arrayfuncs.min_pos((C - Cov) / (AA - corr_eq_dir + tiny))
D:\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:377: RuntimeWarning: overflow encountered in true_divide
  g1 = arrayfuncs.min_pos((C - Cov) / (AA - corr_eq_dir + tiny))
D:\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:377: RuntimeWarning: overflow encountered in true_divide
  g1 = arrayfuncs.min_pos((C - Cov) / (AA - corr_eq_dir + tiny))
D:\Anaconda3\lib\site-packages\sklearn\linear_model\lea

Features sorted by their score:
[(1.0, 'PrevLow'), (0.97999999999999998, 'PrevHigh'), (0.91500000000000004, 'UnemploymentValue'), (0.89500000000000002, 'TBondsOpenValue'), (0.82999999999999996, 'PrevLow2'), (0.79500000000000004, 'PrevHigh4'), (0.79000000000000004, 'PrevVolFrom4'), (0.71999999999999997, 'PrevLow4'), (0.68000000000000005, 'PrevVolFrom'), (0.67000000000000004, 'PrevSP4'), (0.65500000000000003, 'PrevVolTo2'), (0.64000000000000001, 'PrevVolTo'), (0.63500000000000001, 'PrevVolTo3'), (0.60499999999999998, 'PrevHigh2'), (0.59999999999999998, 'PrevVolTo4'), (0.59999999999999998, 'ETHOpenTM1'), (0.58999999999999997, 'PrevSP3'), (0.57999999999999996, 'PrevVolFrom2'), (0.57499999999999996, 'SandPValue'), (0.55500000000000005, 'PrevSP2'), (0.54500000000000004, 'BTCOpenTM7'), (0.52500000000000002, 'PrevVolFrom3'), (0.52000000000000002, 'PrevSP'), (0.48999999999999999, 'PrevLow3'), (0.44500000000000001, 'PrevHigh3'), (0.39500000000000002, 'BTCOpenTM2'), (0.36499999999999999, 'BTCOpen

D:\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:377: RuntimeWarning: overflow encountered in true_divide
  g1 = arrayfuncs.min_pos((C - Cov) / (AA - corr_eq_dir + tiny))
D:\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:381: RuntimeWarning: overflow encountered in true_divide
  g2 = arrayfuncs.min_pos((C + Cov) / (AA + corr_eq_dir + tiny))


In [36]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
 

#use linear regression as the model
lr = LinearRegression()
#rank all features, i.e continue the elimination until the last one
rfe = RFE(lr, n_features_to_select=1)
rfe.fit(train.loc[:, features].values,train.loc[:, 'ETHOpen'].values)
 
print("Features sorted by their rank:")
print(sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), features)))

Features sorted by their rank:
[(1, 'UnemploymentValue'), (2, 'PrevHigh'), (3, 'TBondsOpenValue'), (4, 'PrevLow'), (5, 'BTCOpenTM1'), (6, 'ETHOpenTM1'), (7, 'ETHOpenTM2'), (8, 'PrevLow2'), (9, 'BTCOpenTM2'), (10, 'PrevHigh3'), (11, 'PrevHigh4'), (12, 'BTCOpenTM4'), (13, 'ETHOpenTM4'), (14, 'BTCOpenTM3'), (15, 'PrevLow4'), (16, 'PrevSP3'), (17, 'PrevSP2'), (18, 'PrevSP'), (19, 'SandPValue'), (20, 'PrevSP4'), (21, 'BTCOpenTM6'), (22, 'ETHOpenTM5'), (23, 'PrevHigh2'), (24, 'ETHOpenTM3'), (25, 'ETHOpenTM6'), (26, 'BTCOpenTM5'), (27, 'BTCOpenTM7'), (28, 'ETHOpenTM7'), (29, 'PrevLow3'), (30, 'PrevVolFrom4'), (31, 'PrevVolFrom2'), (32, 'PrevVolFrom'), (33, 'PrevVolFrom3'), (34, 'PrevVolTo2'), (35, 'GDP'), (36, 'PrevVolTo4'), (37, 'PrevVolTo3'), (38, 'PrevVolTo')]


In [188]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso


data = pd.read_csv('C:/Users/JP/Documents/School/Practicum/Github/Practicum/data/dataset2.csv')

df = pd.DataFrame(data)
df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')
df = df.set_index('Date')
df=df[df.index>'2017-05-30']

train=df[df.index<'2017-10-05']

features = ['TBondsOpenValue', 'UnemploymentValue', 'BTCOpenTM1', 'BTCOpenTM2',\
        'BTCOpenTM3',  'BTCOpenTM5', 'BTCOpenTM6', 'BTCOpenTM7','SandPValue',\
            'ETHOpenTM1', 'PrevHigh','PrevHigh2','PrevHigh3','PrevHigh4','PrevLow','PrevLow2','PrevLow3',\
            'PrevLow4','PrevVolTo','PrevVolTo2','PrevVolTo3','PrevVolTo4','PrevVolFrom','PrevVolFrom2','PrevVolFrom3',\
            'PrevVolFrom4','PrevSP','PrevSP2','PrevSP3','PrevSP4',\
         'ETHOpenTM5']


train=train.dropna()


dtrain = xgb.DMatrix(train.loc[:, features].values, \
                     label = train.loc[:, 'ETHOpen'].values)

params = {}
params['booster']  = 'gbtree'
params['objective'] = 'reg:linear'
params['max_depth'] = 8
params['subsample'] = 0.8
params['colsample_bytree'] = 0.8
params['silent'] = 1
params['eval_metric'] = 'rmse'
num_round = 93
eval_list  = [(dtrain,'train')]

train['Date'] = train.index.values

print('Training xgb model:')
bst = xgb.train(params, dtrain, num_round, eval_list)

print('Train Ridge Regression:')
lr = Ridge()
lr.fit(train.loc[:, features].values, \
       train.loc[:, 'ETHOpen'].values)

print('Training Lasso Regression:')
lassoreg = Lasso(alpha=.001,normalize=True, max_iter=1e7)
lassoreg.fit(train.loc[:, features].values,train.loc[:, 'ETHOpen'].values)
 
test2 = df[df.index>='2017-10-05']
test2=test.dropna()
print("7 Features Removed:")
while True:
    dtest2 = xgb.DMatrix(test2[features].values)
    xgb_pred = bst.predict(dtest2)
    lr_pred = lr.predict(test2[features].values)
    lasso_pred = lassoreg.predict(test2.loc[:, features].values)
    test2['ETHOpenRidgexgb2'] = 0.2*xgb_pred+0.8*lr_pred
    test2['ETHOpenRidge2'] = lr_pred
    test2['ETHOpenxgb2'] = xgb_pred
    test2['ETHOpenLasso2'] = lasso_pred


    target = train['ETHOpen']
    
    done = 1
    
    if done:
        print("Prediction: {}".format(test2[['ETHOpen','ETHOpenRidgexgb2','ETHOpenRidge2','ETHOpenxgb2','ETHOpenLasso2']]))
        break
test2 = pd.DataFrame(test2[['ETHOpen','ETHOpenRidgexgb2','ETHOpenRidge2','ETHOpenxgb2','ETHOpenLasso2']])

Training xgb model:
[0]	train-rmse:201.232
[1]	train-rmse:143.494
[2]	train-rmse:102.64
[3]	train-rmse:74.2604
[4]	train-rmse:53.4743
[5]	train-rmse:39.1738
[6]	train-rmse:28.5136
[7]	train-rmse:21.0569
[8]	train-rmse:16.0333
[9]	train-rmse:12.3113
[10]	train-rmse:9.70007
[11]	train-rmse:7.75358
[12]	train-rmse:6.22236
[13]	train-rmse:5.14966
[14]	train-rmse:4.41052
[15]	train-rmse:3.68175
[16]	train-rmse:3.27373
[17]	train-rmse:2.82204
[18]	train-rmse:2.51483
[19]	train-rmse:2.26014
[20]	train-rmse:1.95749
[21]	train-rmse:1.7226
[22]	train-rmse:1.49105
[23]	train-rmse:1.30648
[24]	train-rmse:1.20959
[25]	train-rmse:1.03832
[26]	train-rmse:0.996503
[27]	train-rmse:0.953459
[28]	train-rmse:0.819767
[29]	train-rmse:0.699617
[30]	train-rmse:0.592851
[31]	train-rmse:0.50646
[32]	train-rmse:0.489155
[33]	train-rmse:0.420257
[34]	train-rmse:0.365677
[35]	train-rmse:0.314025
[36]	train-rmse:0.274795
[37]	train-rmse:0.240238
[38]	train-rmse:0.202359
[39]	train-rmse:0.18153
[40]	train-rmse:0.17

In [189]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso


data = pd.read_csv('C:/Users/JP/Documents/School/Practicum/Github/Practicum/data/dataset2.csv')

df = pd.DataFrame(data)
df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')
df = df.set_index('Date')
df=df[df.index>'2017-05-30']

train=df[df.index<'2017-10-05']


features = ['TBondsOpenValue', 'UnemploymentValue', 'BTCOpenTM1', 'BTCOpenTM2',\
        'BTCOpenTM3', 'BTCOpenTM4', 'BTCOpenTM5', 'BTCOpenTM6', 'BTCOpenTM7','SandPValue',\
            'ETHOpenTM1', 'ETHOpenTM2','PrevHigh','PrevHigh2','PrevHigh3','PrevHigh4','PrevLow','PrevLow2',\
            'PrevLow4','PrevVolFrom2',\
            'PrevVolFrom4','PrevSP','PrevSP2','PrevSP3','PrevSP4',\
        'ETHOpenTM3', 'ETHOpenTM4', 'ETHOpenTM5', 'ETHOpenTM6', 'ETHOpenTM7']


train=train.dropna()


dtrain = xgb.DMatrix(train.loc[:, features].values, \
                     label = train.loc[:, 'ETHOpen'].values)

params = {}
params['booster']  = 'gbtree'
params['objective'] = 'reg:linear'
params['max_depth'] = 6
params['subsample'] = 0.8
params['colsample_bytree'] = 0.8
params['silent'] = 1
params['eval_metric'] = 'rmse'
num_round = 50
eval_list  = [(dtrain,'train')]

train['Date'] = train.index.values

print('Training xgb model:')
bst = xgb.train(params, dtrain, num_round, eval_list)

print('Train Ridge Regression:')
lr = Ridge()
lr.fit(train.loc[:, features].values, \
       train.loc[:, 'ETHOpen'].values)

print('Training Lasso Regression:')
lassoreg = Lasso(alpha=.001,normalize=True, max_iter=1e7)
lassoreg.fit(train.loc[:, features].values,train.loc[:, 'ETHOpen'].values)
 
test3 = df[df.index>='2017-10-05']
test3=test3.dropna()

while True:
    dtest3 = xgb.DMatrix(test3[features].values)
    xgb_pred = bst.predict(dtest3)
    lr_pred = lr.predict(test3[features].values)
    lasso_pred = lassoreg.predict(test3.loc[:, features].values)
    test3['ETHOpenRidgexgb3'] = 0.2*xgb_pred+0.8*lr_pred
    test3['ETHOpenRidge3'] = lr_pred
    test3['ETHOpenxgb3'] = xgb_pred
    test3['ETHOpenLasso3'] = lasso_pred


    target = train['ETHOpen']
    
    done = 1
    
    if done:
        print("Prediction: {}".format(test3[['ETHOpen','ETHOpenRidgexgb3','ETHOpenRidge3','ETHOpenxgb3','ETHOpenLasso3']]))
        break
        
test3 = pd.DataFrame(test3[['ETHOpen','ETHOpenRidgexgb3','ETHOpenRidge3','ETHOpenxgb3','ETHOpenLasso3']])

Training xgb model:
[0]	train-rmse:200.88
[1]	train-rmse:142.867
[2]	train-rmse:102.243
[3]	train-rmse:74.2216
[4]	train-rmse:53.2865
[5]	train-rmse:38.6644
[6]	train-rmse:28.9281
[7]	train-rmse:21.8171
[8]	train-rmse:17.1668
[9]	train-rmse:13.7975
[10]	train-rmse:10.9837
[11]	train-rmse:8.59815
[12]	train-rmse:6.95012
[13]	train-rmse:5.7386
[14]	train-rmse:5.06709
[15]	train-rmse:4.14391
[16]	train-rmse:3.44085
[17]	train-rmse:2.84726
[18]	train-rmse:2.43321
[19]	train-rmse:2.17908
[20]	train-rmse:1.98419
[21]	train-rmse:1.79858
[22]	train-rmse:1.59609
[23]	train-rmse:1.41195
[24]	train-rmse:1.21686
[25]	train-rmse:1.08826
[26]	train-rmse:0.964201
[27]	train-rmse:0.856938
[28]	train-rmse:0.768176
[29]	train-rmse:0.703939
[30]	train-rmse:0.630692
[31]	train-rmse:0.558575
[32]	train-rmse:0.496359
[33]	train-rmse:0.438795
[34]	train-rmse:0.403298
[35]	train-rmse:0.349623
[36]	train-rmse:0.308488
[37]	train-rmse:0.263289
[38]	train-rmse:0.242467
[39]	train-rmse:0.208795
[40]	train-rmse:0.

In [142]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from fbprophet import Prophet

data = pd.read_csv('C:/Users/JP/Documents/School/Practicum/Github/Practicum/data/dataset.csv')
#df[['ds','y']] = pd.DataFrame(data[['Date','ETHOpen']])
#df['ds'] = pd.to_datetime(df['ds'],format='%Y-%m-%d')
df = pd.DataFrame(data[['Date','ETHOpen']])
df[['ds','y']] = df 
df = df.set_index('Date')
df = df[df.index>'2017-01-30']
df = df[df.index<'2017-10-05']



m = Prophet(weekly_seasonality = True,yearly_seasonality=True)
m.fit(df) 
future = m.make_future_dataframe(periods=7)

forecast=m.predict(future)
forecast.yhat[-8:]

INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


246    316.174456
247    321.074990
248    322.522106
249    325.066643
250    328.527083
251    331.529887
252    335.372226
253    339.016478
Name: yhat, dtype: float64

In [179]:
from statsmodels.tsa.arima_model import ARIMA
data = pd.read_csv('C:/Users/JP/Documents/School/Practicum/Github/Practicum/data/dataset.csv')
df = pd.DataFrame(data[['Date','ETHOpen']])
df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')
df = df.set_index('Date')
df.tail()
df=df[df.index>'2017-05-30']

traints=np.log(df[df.index<'2017-10-05'])


steps=['2017-10-05','2017-10-06','2017-10-07','2017-10-08','2017-10-09','2017-10-10','2017-10-11','2017-10-12']
for i in steps:
        moving_avg = traints.rolling(window=7,center=False).mean()
        train_moving_avg_diff = traints - moving_avg
        train_moving_avg_diff = train_moving_avg_diff[train_moving_avg_diff.index>'2017-06-05']
        model = ARIMA(train_moving_avg_diff,order=(0,0,3))
        model_fit = model.fit(disp=0)
        yhat = model_fit.forecast(1)[0]
        traints.loc[i] = (yhat+moving_avg[-1:])
        traints.index = pd.to_datetime(traints.index,format='%Y-%m-%d')
        
traints['ETHOpen'] = np.exp(traints['ETHOpen'])        
traints.tail()



,ETHOpen
Date,
2017-10-08,299.875720
2017-10-09,299.153448
2017-10-10,298.835954
2017-10-11,298.861972
2017-10-12,298.920778


In [201]:
#10/5 - 10/12 to be evaluated
data = pd.read_csv('C:/Users/JP/Documents/School/Practicum/Github/Practicum/data/dataset2.csv')

df = pd.DataFrame(data)
df = pd.DataFrame(data[['Date','ETHOpen']])
#df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')
df = df.set_index('Date')

FinalResults = df[df.index>'2017-10-04']
FinalResults['Date'] = FinalResults.index
FinalResults = FinalResults[FinalResults.index<'2017-10-13']
FinalResults['FBProphet'] = forecast.yhat[-8:].values
FinalResults['TSModel'] = traints[traints.index>'2017-10-04']
FinalResults = pd.merge(FinalResults, test1,on='ETHOpen',how='left') 
FinalResults = pd.merge(FinalResults, test2,on='ETHOpen',how='left')
FinalResults = pd.merge(FinalResults, test3,on='ETHOpen',how='left')



FinalResults
#forecast.yhat[-8:]

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,ETHOpen,Date,FBProphet,TSModel,ETHOpenRidgexgb1,ETHOpenRidge1,ETHOpenxgb1,ETHOpenLasso1,ETHOpenRidgexgb2,ETHOpenRidge2,ETHOpenxgb2,ETHOpenLasso2,ETHOpenRidgexgb3,ETHOpenRidge3,ETHOpenxgb3,ETHOpenLasso3
0,291.68,2017-10-05,316.174456,292.967605,290.690407,288.519945,299.372253,289.480750,289.386706,289.571523,288.647430,290.318746,288.508885,287.614065,292.088165,288.632401
1,294.99,2017-10-06,321.074990,295.663091,290.108841,289.656772,291.917114,290.581977,290.411007,289.696498,293.269043,290.722541,289.242730,288.618261,291.740601,289.585514
2,308.33,2017-10-07,322.522106,298.659273,302.111841,303.026359,298.453766,304.075205,300.427788,302.496486,292.152985,303.628138,300.828662,301.954040,296.327148,302.934025
3,311.26,2017-10-08,325.066643,299.875720,305.591141,307.145887,299.372162,308.117369,303.462332,306.974671,289.412964,307.943800,305.285817,305.327798,305.117889,306.317139
4,309.49,2017-10-09,328.527083,299.153448,305.574517,305.701118,305.068115,306.876066,303.471032,304.899346,297.757782,306.160282,304.336472,304.510943,303.638580,305.699212
5,296.95,2017-10-10,331.529887,298.835954,292.145294,292.208023,291.894379,293.340465,291.869936,291.173136,294.657135,292.364516,291.910236,290.882654,296.020569,292.153868
6,298.46,2017-10-11,335.372226,298.861972,301.105066,301.657646,298.894745,302.910014,300.406707,302.581166,291.708862,303.560424,300.611225,300.859490,299.618164,301.945873
7,302.86,2017-10-12,339.016478,298.920778,301.433652,300.949565,303.369995,302.369040,299.476928,301.583914,291.048981,302.877906,301.951909,300.141784,309.192413,301.475188


In [202]:
import plotly.plotly as py
from plotly.graph_objs import *

py.iplot([{
    'x': FinalResults['Date'],
    'y': FinalResults[col],
    'name': col
}  for col in FinalResults.columns], filename='FinalResults')